In [1]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
sys.path.append("../")
sys.path.append("../math_tricks/")
# herramientas para leer pdbs
import read_pdb_tools as rpt
# calculo de distancia
from scipy.spatial.distance import pdist
# libreria de tablas
import pandas as pd
# funciones de click generadas en pandas
import funciones_CLICK as fc
# iteradores
import itertools as it
# cuenta tiempo de ejecucion
import datetime

# NetworkX
import networkx as nx
# filtro distancia
from scipy.spatial import distance

In [2]:
infile = "pdbs/test_short.pdb"
trj = rpt.Trajectory("first")
trj.ReadTraj("%s"%infile, every=1)
trj.PrintTrajInfo()

frame_0 = trj.GetFrame(0)
pdb11 = frame_0.GetResChain("")

frame_1 = trj.GetFrame(1)
pdb22 = frame_1.GetResChain("")

# COMO NO TIENEN CARBONOS ALFA QUITO ESOS RESIDUOS QUE NO TIENEN
print(len(pdb11))
for i,j in enumerate(pdb11):
    
    if (j.resn == 'ACE') or (j.resn == 'NME'):
        pdb11.pop(i)
        
print(len(pdb11))

print(len(pdb22))
for i,j in enumerate(pdb22):
    
    if (j.resn == 'ACE') or (j.resn == 'NME'):
        pdb22.pop(i)
        
print(len(pdb22))

This trajectory file : first
has 2 frames
118
116
118
116


In [3]:
number_elements_clique = 7

In [4]:
# def get_tray_cliques_7(trayectoria):
"""Obtiene cliques de trayectoria"""

# Siempre la proteina 1 es el que se rota y traslada para embonar en la proteina 2
if len(pdb22) < len(pdb11):

    import copy
    pdb1_temp = copy.copy(pdb1)
    pdb2_temp = copy.copy(pdb2)

    pdb11_temp = copy.copy(pdb11)
    pdb22_temp = copy.copy(pdb22)

    pdb1 = pdb2_temp
    pdb2 = pdb1_temp

    pdb11 = pdb22_temp
    pdb22 = pdb11_temp

    del [pdb1_temp]
    del [pdb2_temp]
    del [pdb11_temp]
    del [pdb22_temp]

    print("Intercambio de nombre ya que la proteina 1 es mas grande que la 2")
    print(pdb1.name, len(pdb11))
    print(pdb2.name, len(pdb22))
    print("No te preocupes ya quedo :)")


def get_df_distancias(ref):
    """Funcion para obtener el dataframe de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo
    coord = [res.GetAtom('CA').coord for res in ref]
    lista_residuos = [res.resi for res in ref]

    # calcula distancia y regresa dataframe
    distancias = [ [pdist(np.array([v, av]), metric='euclidean').item() for av in coord] for v in coord]

    # se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=lista_residuos, columns=lista_residuos, data=distancias)
    return(df_da, lista_residuos)


# devuelve tabla e indices de el dataframe de distancias entre atomos de la misma proteina con dth < 10A
df_distancias1, lista_residuos_1 = get_df_distancias(pdb11)
df_distancias2, lista_residuos2 = get_df_distancias(pdb22)

In [5]:
def gen_7_cliques(df_distancias, nombre=False, dth=10, k=7):
    """Genera n-cliques de dataframe de distancias, tomando en cuenta los enlaces menores o iguales
    a dth y forma los k-cliques que elijas 
    valores por default:
    dth=10, k=3"""
# red de distancias completa
    red = nx.from_pandas_adjacency(df_distancias)
#     print("red antes de filtros:",nx.info(red))

    # filtro de distancias
    edgesstrong = [(u, v) for (u, v, d) in red.edges(data=True) if d["weight"] <= dth]

    red = nx.Graph(edgesstrong)
#     print("=="*20)
#     print("red despues de filtros:",nx.info(red))

    cliques_completos = [clq for clq in nx.find_cliques(red) if len(clq) >= k]
    print('numero de cliques maximos encontrados:',len(cliques_completos))
#     print(n_cliques)

    df_maximal_clique = pd.DataFrame(cliques_completos, dtype=int)
    df_maximal_clique['numero_elementos'] = df_maximal_clique.count(1)
    df_maximal_clique.sort_values('numero_elementos', inplace=True)

    if nombre:
        print('guardando red en: %s' % nombre)
        nx.write_gexf(red, nombre+'.gexf')

    return(df_maximal_clique)


# se generan cliques, te devuleve dataframe con cliques de k(numero_de_cliques) y la lista de cliques maximales
cliques1 = gen_7_cliques(df_distancias1,  dth=10, k=number_elements_clique)  # file1[:-4] pal gexf
print('**'*50)
cliques2 = gen_7_cliques(df_distancias2,  dth=10, k=number_elements_clique)  # file2[:-4]
print('**'*50)

# REVISAR DONDE GUARDA EL GEXF Y COLOCARLO EN LA CARPETA Grafos

# se agrega filtro de residuos que pertenecen a cliques de 7 elementos
mc1_7 = cliques1[cliques1.numero_elementos == 7].drop('numero_elementos', 1)
mc2_7 = cliques2[cliques2.numero_elementos == 7].drop('numero_elementos', 1)

residuos_unicos_1 = []
for i in [list(mc1_7[i].unique()) for i in mc1_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_1.append(int(j))

residuos_unicos_2 = []
for i in [list(mc2_7[i].unique()) for i in mc2_7.dropna(1).columns]:
    for j in i:
        residuos_unicos_2.append(int(j))

residuos_unicos_1 = set(residuos_unicos_1)
residuos_unicos_2 = set(residuos_unicos_2)

cliques1_7 = mc1_7[[0,1,2,3,4,5,6]].astype(int).reset_index(drop=True).values
cliques2_7 = mc2_7[[0,1,2,3,4,5,6]].astype(int).reset_index(drop=True).values

# return(cliques1_7, cliques2_7)

numero de cliques maximos encontrados: 108
****************************************************************************************************
numero de cliques maximos encontrados: 94
****************************************************************************************************


In [6]:
print('NUmero de cliques generados FRAME1:',len(cliques1_7))
print('NUmero de cliques generados FRAME2:',len(cliques2_7))

NUmero de cliques generados FRAME1: 77
NUmero de cliques generados FRAME2: 57


In [7]:
# Return the sorted, unique values in `ar1` that are not in `ar2`.
residuos_distintos =  np.setdiff1d(cliques1_7,cliques2_7)
residuos_distintos

array([ 33,  34,  47,  48,  49,  57,  86,  87,  88,  89,  90,  91,  92,
        93,  94, 111, 112])

In [8]:
# Return the sorted, unique values that are in both of the input arrays
residuos_iguales = np.intersect1d(cliques1_7,cliques2_7)
residuos_iguales

array([ 31,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  50,
        51,  52,  53,  54,  55,  56,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  95,  96,  97,  98, 102, 103, 104, 105, 106, 107,
       108, 109, 110, 113, 114, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 132, 133, 134, 135, 136, 137, 139, 140, 141,
       142, 143, 144, 145])

In [36]:
np.isin(cliques1_7, cliques2_7)

array([[ True,  True,  True,  True,  True,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [ True, False,  True, False, False,  True, False],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True, False,  True,  True, False,  True,  True],
       [ True, False,  True,  True, False,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,

In [23]:
mask = [(i in cliques2_7) for i in cliques1_7]
cliques_interseccion_1 = cliques1_7[mask]

mask = [(i in cliques1_7) for i in cliques2_7]
cliques_interseccion_2 = cliques2_7[mask]

In [25]:
cliques_interseccion_1[:5]

array([[ 31,  61,  64, 128,  63,  62, 127],
       [ 90,  96,  97,  80,  91,  92,  93],
       [ 81,  80,  97,  65,  66,  82,  83],
       [ 81,  80,  97,  65,  66,  82,  67],
       [ 81,  80,  97,  96,  82,  66,  67]])

In [29]:
cliques2_7

array([[ 31,  60,  63, 127,  64,  61,  62],
       [ 81,  66,  67,  80,  97,  82,  65],
       [ 81,  66,  67,  80,  97,  82,  96],
       [ 79,  67,  80,  81,  66,  68,  78],
       [ 77,  67,  68,  69,  78, 121,  79],
       [ 77,  67,  68,  69,  78, 121, 122],
       [ 77,  67,  68,  69,  78,  80,  79],
       [ 70,  69,  76, 120, 121, 122,  68],
       [ 95, 109,  96,  97, 108,  80,  67],
       [ 70,  69, 139, 120, 121, 122,  68],
       [ 64,  65,  66, 126, 124, 125, 134],
       [ 63,  64,  65,  82,  83,  84,  85],
       [ 61, 128,  64,  84,  63,  62, 127],
       [126, 134, 125, 135, 124,  65,  35],
       [ 60,  61,  64,  84,  63,  62,  85],
       [ 58, 103, 104, 105, 102,  59,  60],
       [ 56,  55, 105,  36,  35,  37,  54],
       [ 70,  72,  71,  74,  76,  73,  75],
       [ 51, 109,  97, 108,  95,  96, 110],
       [ 97, 108, 107, 106,  98,  99,  54],
       [ 97, 108, 107, 106,  98,  53,  54],
       [126, 128, 129, 131, 132, 133, 127],
       [123, 122,  37, 137,  54,

In [26]:
cliques1_7[:]

array([[ 31,  61,  64, 128,  63,  62, 127],
       [ 90,  96,  97,  80,  91,  92,  93],
       [ 83,  86,  84,  88,  89,  85,  87],
       [ 81,  80,  97,  65,  66,  82,  83],
       [ 81,  80,  97,  65,  66,  82,  67],
       [ 81,  80,  97,  96,  82,  66,  67],
       [ 79, 110,  96,  97, 108, 109,  95],
       [ 79,  93,  96,  97, 109,  95, 110],
       [ 79,  93,  96,  97, 109,  95,  80],
       [ 79,  67,  80,  78,  81,  68,  95],
       [ 79,  67,  80,  78,  81,  68,  66],
       [ 79,  67,  80, 108,  68,  81,  95],
       [ 79,  67,  80, 108,  68,  81,  66],
       [ 79,  94,  96, 109,  93,  95, 110],
       [ 79,  94,  96, 109,  93,  95,  80],
       [126, 134, 135, 125, 136,  66, 124],
       [ 76,  68,  69, 121,  67,  78, 122],
       [ 70,  69, 121,  78, 122,  68,  76],
       [ 70,  69, 121,  77,  76,  68,  78],
       [ 70,  69, 121, 120, 119, 140, 141],
       [ 91,  96,  80,  97,  92,  95,  93],
       [ 70,  69, 121, 120, 122,  68,  76],
       [ 91,  96,  80,  97, 109,

In [102]:
mask = [(i not in cliques2_7) for i in cliques1_7]
cliques1_not_in_cliques2 = cliques1_7[mask]

mask = [(i not in cliques1_7) for i in cliques2_7]
cliques2_not_in_cliques1 = cliques2_7[mask]